# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [1]:
# importing the used packages 
import pandas as pd
import numpy as np
import datetime

# importing package to create plots and setting basic, visual settings
import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"-"})
plt.rcParams.update({'font.size': 10})
import ipywidgets as widgets

# importing the API from DST used to import data
from dstapi import DstApi 

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject



# Read and clean data

1. Data is imported using the API for Danmarks statistik

In [2]:
data = DstApi('EJ55') #EJ55 is data on the pricing of houses
unemp = DstApi('AULK01') #AULK01 is data on the number of unemployed 

## Explore each data set

1. The availble values for each variable is plotted in order to select relevant variables. 

In [3]:
#An overview over the available data. 
tabsum = data.tablesummary(language='en')
display(tabsum)

# Displaying the available values for each variable:
for variable in tabsum['variable name']:
    print(variable+':')
    display(data.variable_levels(variable, language='en'))

Table EJ55: Price index for sales of property by region, category of real property, unit and time
Last update: 2023-03-31T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,OMRÅDE,17,000,All Denmark,11,Province Nordjylland,False
1,EJENDOMSKATE,3,0111,One-family houses,2103,"Owner-occupied flats, total",False
2,TAL,3,100,Index,310,Percentage change compared to same quarter the...,False
3,Tid,124,1992K1,1992Q1,2022K4,2022Q4,True


OMRÅDE:


,id,text
0,000,All Denmark
1,084,Region Hovedstaden
2,01,Province Byen København
3,02,Province Københavns omegn
4,03,Province Nordsjælland
5,04,Province Bornholm
6,085,Region Sjælland
7,05,Province Østsjælland
8,06,Province Vest- og Sydsjælland
9,083,Region Syddanmark


EJENDOMSKATE:


,id,text
0,0111,One-family houses
1,0801,Weekend cottages
2,2103,"Owner-occupied flats, total"


TAL:


,id,text
0,100,Index
1,210,Percentage change compared to previous quarter
2,310,Percentage change compared to same quarter the...


Tid:


,id,text
0,1992K1,1992Q1
1,1992K2,1992Q2
2,1992K3,1992Q3
3,1992K4,1992Q4
4,1993K1,1993Q1
...,...,...
119,2021K4,2021Q4
120,2022K1,2022Q1
121,2022K2,2022Q2
122,2022K3,2022Q3


We are only interested in some of 

1. A param dictionary is defined in order to detaile the data we want

In [4]:
# Getting an overview of the underlying code that determines which variables and subset of data we import via the API
params = data._define_base_params(language='en')
params

{'table': 'ej55',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'OMRÅDE', 'values': ['*']},
  {'code': 'EJENDOMSKATE', 'values': ['*']},
  {'code': 'TAL', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

1. We select the data we want. We only want data for "All Denmark" and indexed values, and percentage change compared to previous quarter.

In [5]:
# Using the format printed above to specify which subsets of the available dataset we want to import
params = {'table': 'ej55',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'OMRÅDE', 'values': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']},
  {'code': 'EJENDOMSKATE', 'values': ['0111']},
  {'code': 'TAL', 'values': ['100']}, # with the key 'code' we choose the desired variable and with the key 'values' we choose what subset of the dataset for the given variable we want to include
  {'code': 'Tid', 'values': ['*']}]} # ['*'] includes all the available data

1. Data is sorted and the index is reset. 
2. Columns are renamed.

In [6]:
sales_api = data.get_data(params=params) # retrieving the specified subset of the dataset from DST
sales_api.sort_values(by=['OMRÅDE', 'TID', 'EJENDOMSKATE'], inplace=True) # sorting the values
sales_api.reset_index(inplace = True, drop = True) # resetting the initial index, so it fits the new dataset
sales_api.rename(columns = {'OMRÅDE':'PROVINCE', 'EJENDOMSKATE':'CATEGORY', 'TAL':'UNIT', 'TID':'TIME', 'INDHOLD':'SALES_INDEX'}, inplace=True) # renaming columns

1. Missing values are replaced with NaN.

In [7]:
sales_api = sales_api.replace('..', np.nan) # replacing all the missing data (denoted with '..' by DST) with NaN-values

1. Values types are replaced. 

In [8]:
sales_api.info() # assesing the types of data in the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1364 entries, 0 to 1363
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PROVINCE     1364 non-null   object
 1   CATEGORY     1364 non-null   object
 2   UNIT         1364 non-null   object
 3   TIME         1364 non-null   object
 4   SALES_INDEX  1308 non-null   object
dtypes: object(5)
memory usage: 53.4+ KB


1. The value variable is changed to er float type variable. 

In [9]:
sales_api.SALES_INDEX = sales_api.SALES_INDEX .astype('float') # changing the column SALES_INDEX from object to float
sales_api.info() # displaying the new types of variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1364 entries, 0 to 1363
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PROVINCE     1364 non-null   object 
 1   CATEGORY     1364 non-null   object 
 2   UNIT         1364 non-null   object 
 3   TIME         1364 non-null   object 
 4   SALES_INDEX  1308 non-null   float64
dtypes: float64(1), object(4)
memory usage: 53.4+ KB


In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

1. We make an interactive plot

In [17]:
def plot_value(df, province): 
    I = (df['PROVINCE'] == province) # creating the plot for the variable 'PROVINCE', thus allowing us to choose the desired province in the plot later on 
    ax=df.loc[I,:].plot(x='TIME', y='SALES_INDEX', legend=False) # specifying the x- and y-variables for the plot

widgets.interact(plot_value, # creating an interactive plot
    df = widgets.fixed(sales_api),
    province = widgets.Dropdown(description='Province', #creating drop-down widget that allows us to choose the desired province
                                    options=sales_api.PROVINCE.unique(), 
                                    value='Province Byen København'), # initial province will be 'Byen København'
)


interactive(children=(Dropdown(description='Province', index=1, options=('Province Bornholm', 'Province Byen K…

<function __main__.plot_value(df, province)>

Explain what you see when moving elements of the interactive plot around. 

# Merge with data on gross unemployment

Now we wish to examine the correlation within provinces between gross unemployment and prices on one-family houses. Theory suggests that the gross unemployment in a province is a determinant of the housing prices, and that they are negatively correlated. The intuition is that when the unemployment rate increases in a province, then it becomes less attractive to move to this province as the risk of unemployment, all else equal, is higher in this area. Therefore, the demand of houses in the given province decreases implying that the housing prices decrease.

First, we collect data on gross unemployment within provinces in Denmark. This is done by using the API from Statistics Denmark and importing the register *AULK01*. 

Just like before, we then clean this dataset and select the relevant information from the dataset. To do this, we need to get an overview of the dataset:

In [11]:
#An overview over the available data. 
tabsum_unemp = unemp.tablesummary(language='en')
display(tabsum)

# The available values for each variable:
for variable in tabsum_unemp['variable name']:
    print(variable+':')
    display(unemp.variable_levels(variable, language='en'))

Table AULK01: Full-time unemployed persons by region, type of benefits, unemployment insurance fund, age, sex and time
Last update: 2023-03-17T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,OMRÅDE,17,000,All Denmark,11,Province Nordjylland,False
1,EJENDOMSKATE,3,0111,One-family houses,2103,"Owner-occupied flats, total",False
2,TAL,3,100,Index,310,Percentage change compared to same quarter the...,False
3,Tid,124,1992K1,1992Q1,2022K4,2022Q4,True


OMRÅDE:


,id,text
0,000,All Denmark
1,084,Region Hovedstaden
2,01,Province Byen København
3,101,Copenhagen
4,147,Frederiksberg
...,...,...
112,787,Thisted
113,820,Vesthimmerlands
114,851,Aalborg
115,998,Unknown municipality


YDELSESTYPE:


...

,id,text
0,TOT,"Age, total"
1,16-24,16-24 years
2,25-29,25-29 years
3,30-34,30-34 years
4,35-39,35-39 years
5,40-44,40-44 years
6,45-49,45-49 years
7,50-54,50-54 years
8,55-59,55-59 years
9,6099,60 year and over


KØN:


,id,text
0,TOT,Total
1,M,Men
2,K,Women


Tid:


,id,text
0,2007K1,2007Q1
1,2007K2,2007Q2
2,2007K3,2007Q3
3,2007K4,2007Q4
4,2008K1,2008Q1
...,...,...
59,2021K4,2021Q4
60,2022K1,2022Q1
61,2022K2,2022Q2
62,2022K3,2022Q3


Now, we select the relevant variables for this analysis.

In [12]:
# Getting an overview of the underlying code that determines which variables and subset of data we import via the API
params_unemp = unemp._define_base_params(language='en')
params_unemp 

{'table': 'aulk01',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'OMRÅDE', 'values': ['*']},
  {'code': 'YDELSESTYPE', 'values': ['*']},
  {'code': 'AKASSE', 'values': ['*']},
  {'code': 'ALDER', 'values': ['*']},
  {'code': 'KØN', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

In [13]:
# Using the format printed above to specify which subsets of the available dataset we want to import
params_unemp = {'table': 'aulk01',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'OMRÅDE', 'values': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']},
  {'code': 'YDELSESTYPE', 'values': ['TOT']}, # with the key 'code' we choose the desired variable and with the key 'values' we choose what subset of the dataset for the given variable we want to include
  {'code': 'AKASSE', 'values': ['TOT']},
  {'code': 'ALDER', 'values': ['TOT']},
  {'code': 'KØN', 'values': ['TOT']},
  {'code': 'Tid', 'values': ['*']}]} # ['*'] includes all the available values


In [14]:
unemp_api = unemp.get_data(params=params_unemp) # retrieving the specified subset of the dataset from DST
unemp_api.sort_values(by=['OMRÅDE', 'TID'], inplace=True) # sorting the values
unemp_api = unemp_api.drop(columns = ['YDELSESTYPE', 'AKASSE', 'ALDER', 'KØN']) # dropping unwanted columns
unemp_api.reset_index(inplace = True, drop = True) # resetting the index, so it fits the new dataset
unemp_api.rename(columns = {'OMRÅDE':'PROVINCE','TID':'TIME', 'INDHOLD':'GROSS UNEMPLOYMENT'}, inplace=True) # renaming columns
unemp_api.head(5) # displaying the first five values of the dataset

,PROVINCE,TIME,GROSS UNEMPLOYMENT
0,Province Bornholm,2007Q1,1930
1,Province Bornholm,2007Q2,1364
2,Province Bornholm,2007Q3,1060
3,Province Bornholm,2007Q4,1459
4,Province Bornholm,2008Q1,1572


Now, we wish to merge gross unemployment data within provinces on the dataset of housing prices. Here, we merge on province and time, such that we for each province in each quarter now also have information on the number of gross employed. We have data on housing prices from 1992Q1 and onwards, while we only have data on gross unemployment from 2007Q1. As all observations on housing prices before 2007Q1 cannot be linked to the gross unemployment in the given period, these observations are irrelvant, and we therefore use the *inner join*-method to merge. This only method only includes the matches between the two datasets.


In [15]:
sales_with_unemp = pd.merge(sales_api, unemp_api, on = ['PROVINCE', 'TIME'], how = 'inner') # Performing an inner merge of the two datasets (keeping only data for which there are observations for both variables in the dataset)
sales_with_unemp.sample(10) # displaying a sample of 10 random values in the dataset

,PROVINCE,CATEGORY,UNIT,TIME,SALES_INDEX,GROSS UNEMPLOYMENT
683,Province Østsjælland,One-family houses,Index,2017Q4,94.5,4047
593,Province Østjylland,One-family houses,Index,2011Q2,95.0,21254
5,Province Bornholm,One-family houses,Index,2008Q2,119.1,998
610,Province Østjylland,One-family houses,Index,2015Q3,99.8,16005
207,Province Københavns omegn,One-family houses,Index,2010Q4,82.9,14067
145,Province Fyn,One-family houses,Index,2011Q2,99.1,15334
228,Province Københavns omegn,One-family houses,Index,2016Q1,99.3,12006
346,Province Nordsjælland,One-family houses,Index,2013Q3,74.4,8572
655,Province Østsjælland,One-family houses,Index,2010Q4,77.0,5724
320,Province Nordsjælland,One-family houses,Index,2007Q1,103.0,6697


Now that the dataset is ready, we can examine the correlation between gross unemployment and prices on one-family houses within each province. We do this by constructing a binned scatterplot for each province. The binned scatterplot groups the unemployment rates into 10 equal-sized bins for each province and plots the mean of the associated housing prices within each bin. 

We construct the binned scatter plot using the package *binsreg*, which can be installed writing "pip install binsreg" in the prompt.

In [16]:
# Import binsreg
import binsreg

def binscatter(df,province):
    I =df[df['PROVINCE'] == province]
    binsreg.binsreg('SALES_INDEX', 'GROSS UNEMPLOYMENT', data=I, 
                    nbins=10, #specify 10 bins 
                    polyreg=1, #create linear fitted line      
    )

    
widgets.interact(binscatter, # creating interactive widget letting us choose the desired province
    df = widgets.fixed(sales_with_unemp),
    province = widgets.Dropdown(description='Province', 
                                    options=sales_with_unemp.PROVINCE.unique(), 
                                    value='Province Byen København'),
)



interactive(children=(Dropdown(description='Province', index=1, options=('Province Bornholm', 'Province Byen K…

<function __main__.binscatter(df, province)>

From the interactive figure above, we clearly observe a negative correlation between gross unemployment rate and housing prices in all of the provinces. This results supports the theoretical suggestions that the gross unemployment rate in a province is a determinant of the housing prices in the province. However, it is important to emphasize that these results are only correlations, hence we cannot give the results any causal interpretations. In principle, the negative correlation could be due to reverse causality or omitted variables correlating with both gross unemployment and the housing prices (e.g. the interest rates).